### Import Libraries

In [55]:
import os
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

### Import Useful Functions

In [10]:
parent_directory = os.path.join(os.environ["BLACK_BOX"])
validations_path = os.path.join(parent_directory, "experiments/results/validation_checkpoints")
sys.path.append(parent_directory)

from experiments.utils.validation_utils import extract_results_csv

### Get Algorithm Validation Results Paths

In [11]:
# validation algorithm resultant folders located in ./experiments/results/validation_checkpoints folder
folder_name_1 = "ce_search_PPOTrainer_highway_environment:highway-environment-v0_2022-09-08_19-40-43iwf79ufp_Agent2_ce_Chkpt300"

### Read Verification Results

In [15]:
# read results.csv file inside given validation folder
df_1 = extract_results_csv(os.path.join(validations_path, folder_name_1, "results.csv"))
df_1

,collision,impossible,episode_length,episode_min_ttc,reward,statistics/ego_speeds,statistics/ego_accels,statistics/ego_jerks,statistics/ego_actions,statistics/ego_rewards,statistics/front_positions,statistics/front_speeds,statistics/tgap,statistics/ttc,config/ego_v1,config/front_v1,config/front_v2,config/delta_dist
0,False,False,249,2.739833,1213.939199,"[33.46259464031343, 29.46259464031344, 25.4625...","[-3.999999999999986, -3.999999999999986, -3.99...","[-3.999999999999986, 0.0, 0.0, 0.0548501014709...","[array([-1.], dtype=float32), array([-1.], dty...","[-0.004229096968547008, 1.0960393069316017, 0....","[15.300314215961365, 11.389661056920446, 10.76...","[-5.260735394416223, -2.1650154822689913, 1.31...","[0.45723620084040334, 0.386580249640796, 0.422...","[-2.9083928357759308, -5.260751501633064, 8.17...",37.462595,29.886229,25.986542,21.926723
1,False,False,249,16.790449,1422.742960,"[15.26739620271648, 19.26739620271647, 23.2673...","[4.0000000000000036, 3.9999999999999893, 3.999...","[4.0000000000000036, -1.4210854715202004e-14, ...","[array([1.], dtype=float32), array([1.], dtype...","[-3.748653083011044, -0.6315458776270639, 0.33...","[55.83408936154753, 58.57234459819345, 59.8826...","[3.325358225671728, 1.9874952378553417, 0.4385...","[3.657077608350602, 3.0399704029666155, 2.5736...","[16.79044887104084, 29.470580954552286, 136.55...",11.267396,15.779386,34.265586,51.846002
2,False,False,249,26.268648,890.425319,"[24.25655475396593, 28.256554753965915, 25.206...","[3.999999999999986, 3.999999999999986, -3.0499...","[3.999999999999986, 0.0, -7.049911022186279, -...","[array([1.], dtype=float32), array([1.], dtype...","[-1.7222684282938627, 1.48610807498611, 1.4977...","[66.81077253390136, 57.811837088941004, 47.184...","[-6.555609187163967, -11.759719894703547, -9.1...","[2.754337772539611, 2.0459612692596294, 1.8718...","[-10.19137485360928, -4.916085454903078, -5.14...",20.256555,21.778907,15.688111,69.335963
3,False,False,249,10.743792,1549.015136,"[19.3267884371694, 23.326788437169387, 27.3267...","[3.9999999999999893, 3.999999999999986, 3.9999...","[3.9999999999999893, -3.552713678800501e-15, 0...","[array([1.], dtype=float32), array([1.], dtype...","[-4.5093083145301875, -1.5077821033252894, -0....","[95.18751408201197, 103.18914071898325, 109.11...","[8.859779018461115, 6.903363809047715, 4.66978...","[4.9251568691763685, 4.423630657971461, 3.9928...","[10.74379183968223, 14.947682470474117, 23.365...",15.326788,25.887366,38.788527,85.372520
4,False,False,249,21.806098,825.078074,"[23.742958006924795, 27.74295800692478, 31.742...","[3.999999999999986, 3.999999999999986, 3.99999...","[3.999999999999986, 0.0, 0.0, -7.0090982913970...","[array([1.], dtype=float32), array([1.], dtype...","[-2.555028855410509, 0.4768455097754525, 1.688...","[83.64406890045376, 82.97988982850036, 72.3488...","[3.8358209280466227, -6.164179071953377, -15.6...","[3.5228986062761156, 2.991024241090145, 2.2792...","[21.806097972365208, -13.46160684883158, -4.61...",19.742958,33.578779,13.511508,74.308248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,False,False,249,1.231904,1456.869790,"[24.44043465473945, 20.440434654739466, 16.528...","[-3.999999999999986, -3.999999999999986, -3.91...","[-3.999999999999986, 0.0, 0.08779621124265446,...","[array([-1.], dtype=float32), array([-1.], dty...","[-2.1949805727672653, -1.312949725754447, 0.29...","[7.520056052343996, 5.284494108684761, 9.57963...","[-5.246429144726726, 1.4118391626771682, 7.776...","[0.30768900314936454, 0.2585312696443451, 0.57...","[-1.4333638197066514, 3.743012432654449, 1.231...",28.440435,16.386524,35.197178,16.500570
8188,True,False,1,100.000000,-12.242039,"[24.672346177035905, 14.612886326753262]","[-3.999999999999986, -10.059459850282643]","[-3.999999999999986, -6.059459850282657]","[array([-1.], dtype=float32), array([-1.], dty...","[-2.242039144178123, -10.0]","[7.417842701692706, 5.0]","[-5.78425010794329, -1.1626113712820931]

### Print General Statistics

In [13]:
n_samples = df_1.shape[0]
print("Number of Samples :\t", df_1.shape)

n_collisions = df_1["collision"].value_counts()[True]
print("Number of Collisions :\t", n_collisions)

Number of Samples :	 (8192, 18)
Number of Collisions :	 2399


### Extract Useful Columns

In [34]:
ego_v1 = df_1["config/ego_v1"].to_numpy()
front_v1 = df_1["config/front_v1"].to_numpy()
front_v2 = df_1["config/front_v2"].to_numpy()
delta_dist = df_1["config/delta_dist"].to_numpy()
min_ttc = df_1["episode_min_ttc"].to_numpy()

In [73]:
front_v1_str = "MIO-VEL-1"
front_v1_str

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 1-5: truncated \uXXXX escape (1313549488.py, line 1)

In [86]:
front_v1_str = "v\u00bim"
front_v1_str

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 1-5: truncated \uXXXX escape (1054838200.py, line 1)

In [74]:
foo = "some fancy label and $a < \\gamma$"
foo

'some fancy label and $a < \\gamma$'

In [77]:
configs_df = df_1[["config/ego_v1", "config/front_v1", "config/front_v2", "config/delta_dist", "episode_min_ttc"]]
configs_df = configs_df.rename(columns={
                                        "config/ego_v1": "EGO initial velocity",
                                        "config/front_v1": "MIO initial velocity",
                                        "config/front_v2": "MIO target velocity",
                                        "config/delta_dist": "MIO distance",
                                        "episode_min_ttc": "\u03C4 (s)"})
configs_df

,EGO initial velocity,MIO initial velocity,MIO target velocity,MIO distance,τ (s)
0,37.462595,29.886229,25.986542,21.926723,2.739833
1,11.267396,15.779386,34.265586,51.846002,16.790449
2,20.256555,21.778907,15.688111,69.335963,26.268648
3,15.326788,25.887366,38.788527,85.372520,10.743792
4,19.742958,33.578779,13.511508,74.308248,21.806098
...,...,...,...,...,...
8187,28.440435,16.386524,35.197178,16.500570,1.231904
8188,28.672346,16.083754,34.455514,16.934123,100.000000
8189,28.150689,16.541633,34.676004,16.464698,1.338618
8190,28.008922,16.603475,35.202417,16.672820,1.412262


### Remove Unnecessary (High TTC) Samples

In [79]:
configs_df_risky = configs_df[configs_df["\u03C4 (s)"] < 2.0]
configs_df_risky

,EGO initial velocity,MIO initial velocity,MIO target velocity,MIO distance,τ (s)
17,22.354286,14.708907,38.990787,13.305625,1.864013
79,28.507972,10.033621,24.949037,31.514751,1.372187
260,29.421652,18.889108,36.899301,18.151568,1.762868
263,27.976238,24.288291,38.906435,11.428027,1.857140
306,16.292103,36.077991,32.539674,11.310431,1.964062
...,...,...,...,...,...
8186,28.317654,16.053212,34.774260,16.880528,1.230800
8187,28.440435,16.386524,35.197178,16.500570,1.231904
8189,28.150689,16.541633,34.676004,16.464698,1.338618
8190,28.008922,16.603475,35.202417,16.672820,1.412262


In [63]:
fig = px.scatter_3d(configs_df_risky,
                    x="EGO initial velocity (m/s)", y="MIO initial velocity (m/s)", z="MIO target velocity (m/s)",
                    color="MIO distance (m)",
                    size_max=1)
fig.show()

In [80]:
fig = px.scatter_matrix(configs_df_risky,
                       dimensions=["EGO initial velocity",
                                   "MIO initial velocity",
                                   "MIO target velocity",
                                   "MIO distance"],
                       color="\u03C4 (s)")
fig.show()

In [60]:
"\u03C4 (s)"

'τ (s)'